::Django Signal Topic Assessment::

Question 1:
By default are django signals executed synchronously or asynchronously? Please support your answer with a code snippet that conclusively proves your stance. The code does not need to be elegant and production ready, we just need to understand your logic.
 
Answer:

In [ ]:
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.db import models

class MyModel(models.Model):
    name = models.CharField(max_length=100)

@receiver(post_save, sender=MyModel)
def my_signal_handler(sender, instance, **kwargs):
    print("Signal received for:", instance.name)

# Creating an instance of MyModel
obj = MyModel.objects.create(name="Test")
print("Object created")  # This will be displayed after the signal handler finishes


Explanation for above:
when a signal is sent, the connected receivers are executed one by one in the same order in which they
were connected, and the signal sender waits for all receivers to finish before continuing execution.
A synchronous signal is sent and controlled immediately when the event occurs, before the
sender resumes its execution. After the sender completes its execution, an asynchronous
signal is sent and handled later. Depending on the use case, Django provides both types of
signals.

The code defines a Django model MyModel with a name field.
A signal handler function, my_signal_handler, is connected to the post_save signal of MyModel using the @receiver decorator.
When an instance of MyModel is created (obj = MyModel.objects.create(name="Test")), the post_save signal is fired, and the signal handler is called. The handler prints a message that shows the name of the instance.
The message "Object created" will only appear after the signal handler has been executed, demonstrating the synchronous nature of signals.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Question 2: 
Do django signals run in the same thread as the caller? Please support your answer with a code snippet that conclusively proves your stance. The code does not need to be elegant and production ready, we just need to understand your logic.

Answer:
Django signals typically run in the same thread as the caller. Since signals are executed synchronously by default, the signal handlers are executed in the same thread that triggers the signal. To prove this, we can use Python’s threading module to check the thread identifiers of both the signal caller and the signal handler.

In [ ]:
import threading
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.db import models

class MyModel(models.Model):
    name = models.CharField(max_length=100)

@receiver(post_save, sender=MyModel)
def my_signal_handler(sender, instance, **kwargs):
    print("Signal received in thread:", threading.get_ident())

# Creating an instance of MyModel
print("Object creation in thread:", threading.get_ident())
obj = MyModel.objects.create(name="Test")


Explanation For Above code:
We import threading.get_ident() to retrieve the unique identifier of the current thread.
When creating an instance of MyModel, we print the thread identifier where the instance creation occurs.
In the signal handler my_signal_handler, we also print the thread identifier when the signal is triggered.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Question 3:
By default do django signals run in the same database transaction as the caller? Please support your answer with a code snippet that conclusively proves your stance. The code does not need to be elegant and production ready, we just need to understand your logic.

Answer:
Django signals do not run in the same database transaction as the caller, unless they are explicitly wrapped in a transaction. This means that signals, such as post_save, are executed after the database transaction has been committed, and the changes have been saved to the database. If the transaction fails or is rolled back, the signal handlers will not be executed unless the signal itself is triggered within a transaction block.
To prove this, we can use a code snippet that demonstrates how signals behave when a database transaction is rolled back. We'll use Django's transaction.atomic() to manage the transaction and simulate a rollback.

In [ ]:
from django.db import models, transaction
from django.db.models.signals import post_save
from django.dispatch import receiver

class MyModel(models.Model):
    name = models.CharField(max_length=100)

@receiver(post_save, sender=MyModel)
def my_signal_handler(sender, instance, **kwargs):
    print("Signal received for:", instance.name)

# Simulating a rollback within a transaction
try:
    with transaction.atomic():
        print("Creating an instance of MyModel...")
        obj = MyModel.objects.create(name="Test")
        print("Instance created, but raising an exception to rollback transaction.")
        raise Exception("Rolling back transaction")
except Exception as e:
    print("Transaction rolled back:", str(e))

Explanation for above code:
We define a simple model MyModel and connect a signal handler my_signal_handler to the post_save signal.
Inside the transaction.atomic() block, we attempt to create an instance of MyModel.
After the instance is created, we force an exception to trigger a rollback, simulating a failed transaction.
The post_save signal will still be called, but the creation of the object will not persist in the database due to the rollback.

Conclusion:
The signal handler my_signal_handler is still called after the object creation, but since the transaction is rolled back, the object will not persist in the database. This demonstrates that, by default, Django signals are not automatically tied to the transaction lifecycle, meaning they are called even if the transaction fails. If you want signals to be triggered only after a successful commit, you should use the transaction.on_commit() hook. 
example code:

In [ ]:
from django.db import transaction

@receiver(post_save, sender=MyModel)
def my_signal_handler(sender, instance, **kwargs):
    transaction.on_commit(lambda: print("Signal received after commit for:", instance.name))

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

::Custom Class Topic Assessment::

Description: You are tasked with creating a Rectangle class with the following requirements:

An instance of the Rectangle class requires length:int and width:int to be initialized.
We can iterate over an instance of the Rectangle class 
When an instance of the Rectangle class is iterated over, we first get its length in the format: {'length': <VALUE_OF_LENGTH>} followed by the width {width: <VALUE_OF_WIDTH>}


Answer:

In [ ]:
class Rectangle:
    def __init__(self, length: int, width: int):
        self.length = length
        self.width = width
        self._data = [{'length': self.length}, {'width': self.width}]
        self._index = 0  # Initialize the index for iteration
    
    def __iter__(self):
        self._index = 0  # Reset index when iteration starts
        return self

    def __next__(self):
        if self._index < len(self._data):
            result = self._data[self._index]
            self._index += 1
            return result
        else:
            raise StopIteration  # Raise exception when all data is iterated over

# Example usage
rect = Rectangle(10, 5)
for item in rect:
    print(item)

Explanation For Above Code:

__init__() method: The class is initialized with length and width. These values are stored in the _data list in the required format ({'length': <VALUE>} and {'width': <VALUE>}), which the iterator will return.
__iter__() method: This method resets the iteration by setting _index to 0. It also returns the instance itself as an iterator.
__next__() method: This method returns the next value in the _data list. If all items have been returned, it raises the StopIteration exception to stop the iteration.